In [ ]:
import subprocess
download_command = "wget --directory-prefix downloads https://cricsheet.org/register/people.csv"
subprocess.run(download_command.split(" "))

In [ ]:
from etl.spark.spark_session_helper import spark
all_players_df = spark.read.csv("downloads/people.csv", header=True)
all_players_df.registerTempTable("all_players")
all_players_df.toPandas()